In [1]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from jupyterplot import ProgressPlot
from tqdm.notebook import tqdm
import numpy as np

from Define import *
import NUS_WIDE as nus
import TenNet as TenNet
import PredictionModel as P
from TripletLossFunc import TripletLossFunc
from Word2Vec import *

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
getTenNetFromFile = False
getDecoderFromFile = False
test = False
evalue = True
#NUS81 NUS1k COCO2017
dataSet = "NUS81"
BATCH_SIZE = 64

In [4]:
if test or device == torch.device('cpu'):
    Number_Of_Images_Train = 100
    Number_Of_Images_Valid = 100
    N_Epochs = 1
    N_Epochs_Decoder = 1
    BATCH_SIZE = 32

#NUS_WIDE
if dataSet == "NUS81": 
    train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_81, number=Number_Of_Images_Train, min_tag_num=1)
    valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_81,  number=Number_Of_Images_Valid, min_tag_num=1)
    test_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_81, min_tag_num=2)
elif dataSet == "NUS1k":
    BATCH_SIZE = 20
    train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_1k, number=Number_Of_Images_Train, min_tag_num=1)
    valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_1k,  number=Number_Of_Images_Valid, min_tag_num=1)
    test_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_1k, min_tag_num=2)
elif dataSet == "COCO2017":
    print("COCO")

batch_size = BATCH_SIZE
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, drop_last=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, drop_last=True, batch_size=batch_size)
#test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [5]:
if test or device == torch.device('cpu'):
    image_model = models.vgg11(pretrained=True).to(device)
    Word2Vec_wv = [[0 for i in range(Word_Dimensions)] for j in range(len(valid_data.get_tag_list()))]
else:
    image_model = models.vgg16(pretrained=True).to(device)
    Word2Vec_wv = compute_word_matrix(train_data.image_tags, len(train_data.tag_list)) 

#tag_model = TenNet.TenNet_Tag(valid_data.get_tag_list(), in_channel=2, additional_matrix=Word2Vec_wv).to(device)
#tag_model = TenNet.TenNet_Tag_V2(valid_data.get_tag_list(), in_channel=2, additional_matrix=Word2Vec_wv).to(device)

triplet_loss = TripletLossFunc(Margin_Distance)
#triplet_loss = TripletLossFunc(Margin_Distance,hybrid=True,beta=0.05,t1=0,t2=0.005*Single_Distance*Feature_Dimensions)

In [6]:
tag_list = valid_data.get_tag_list()
filters=[3, 4, 5]
filter_num=[100, 100, 100]
tag_model = TenNet.TenNet_Tag_s1(tag_list, filters=filters, filter_num=filter_num, in_channel=2, additional_matrix=Word2Vec_wv).to(device)

In [7]:
if getTenNetFromFile:
    name = "../SavedModelState/IT_model_" + str(Margin_Distance) +".ckpt"
    TenNet.getTenModel(tag_model, image_model, name=name)
else:
    lr = 0.001
    gamma = 0.9
    n_epochs = 10
    Lambda = 0.1
    ten_res = TenNet.run(image_model,tag_model,train_loader,valid_loader,triplet_loss,lr=lr,gamma=gamma,n_epochs=n_epochs,print_log=True,Lambda=Lambda)

  0%|          | 0/1 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\deng_\code\python\TENN\Triplet-Embeddi

epoch:0:
 1-train dataset train model
loss: 3008.58,  IT_pos_dis: 221.06,  II_pos_dis: 104.80,  IT_neg_dis: 218.60,  II_neg_dis: 104.46.
 2-valid dataset evalue model
loss: 1274.13,  IT_pos_dis: 93.98,  II_pos_dis: 14.33,  IT_neg_dis: 93.16,  II_neg_dis: 15.02.


In [ ]:
if not getTenNetFromFile:
    res = ten_res
    TenNet.printLossProgressPlot(res, n_epochs)
    TenNet.printDistanceProgressPlot(res, n_epochs, train=True)
    TenNet.printDistanceProgressPlot(res, n_epochs, train=False)
else:
    TenNet.test(image_model, tag_model, valid_loader, triplet_loss, Lambda=0.1)

TenNet.evalue(train_data, image_model, tag_model, k=3)
TenNet.evalue(valid_data, image_model, tag_model, k=3)
TenNet.evalue(test_data, image_model, tag_model, k=3)

In [6]:
###########################################
############################################
###########################################
##########################################
#######################################
###########################################

NameError: name 'torch' is not defined

In [ ]:
if not getDecoderFromFile:
    res = hybrid_res
    P.printLossProgressPlot(res, n_epochs)
    P.printTagNumProgressPlot(res, n_epochs)
    P.printAccuracyProgressPlot(res, n_epochs)

In [ ]:
predict_model = P.TagDecoder(train_data.get_tag_num()).to(device)
mod = P.LossModel.PredictModel
predict_res =P.run(predict_model,mod,tag_model,image_model,train_loader,valid_loader,loss_funk,threshold,n_epochs,test,getDecoderFromFile)

In [ ]:
res = predict_res
#P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)

In [ ]:
decoder_model = P.TagDecoder(train_data.get_tag_num()).to(device)
mod = P.LossModel.DecoderModel
decoder_res =P.run(decoder_model,mod,tag_model,image_model,train_loader,valid_loader,loss_funk,threshold,n_epochs,test,getDecoderFromFile)

In [ ]:
res = decoder_res
#P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)